# setup to create and ingest 2 gri dwarf catalogs

In [1]:
import numpy as np
import artpop
import astropy.units as u
from astropy.table import Table
from astropy.coordinates import SkyCoord
from dustmaps.sfd import SFDQuery
import numpy as np
import pandas as pd
import sys
sys.path.append("/home/p/pferguso/u/repos/psferguson/starlink/python/")
from synthpop import adopt_a_cat, massage_the_cat
import utils as sl_utils
from lsst.daf.butler import Butler, DimensionUniverse, DatasetType, CollectionType
from lsst.daf.butler.registry import MissingCollectionError
import lsst.source.injection as si

Overriding default configuration file with /opt/lsst/software/stack/stack/miniconda3-py38_4.9.2-7.0.1/Linux64/dustmaps_cachedata/gbb0a0c949e+81bc2a20b4/config/.dustmapsrc


In [2]:
repo="/repo/main"
collection='HSC/runs/RC2/w_2023_32/DM-40356'
butler = Butler(repo, collections=collection)
registry = butler.registry

In [3]:
bands = ["g","r","i"]

In [4]:
dataid = {}
for band in bands:
    dataid[band] = {'band': band, 'skymap': 'hsc_rings_v1', 'tract': 9615, 'patch': 3}

In [5]:
coadd = {}
for band in bands: 
    coadd[band] = butler.get('deepCoadd_calexp', dataId=dataid[band])

In [6]:
wcs = coadd["g"].getWcs()
bbox = coadd['g'].getBBox()

## Make the catalog to inject

In [7]:
cat0 = adopt_a_cat(wcs, bbox, 1000.0, 1000.0, r_scale=100.0, random_seed=69)

In [8]:
writeable_butler =  Butler(repo, writeable=True)

In [9]:
injection_cats={}
si_input_collections = []
for lim in [27,29,30,31,33]:
    injection_cats[lim]={}
    si_input_collections.append(f"u/pferguso/full_dwarf_test/2_lim_{lim}")
    for band in bands: 
        injection_cats[lim][band] = massage_the_cat(cat0, float(lim), band, 1000.0, 1000.0, wcs, bbox, r_scale=100.0)
    

In [10]:
for si_input_collection in si_input_collections:
    try:
        writeable_butler.removeRuns([si_input_collection])
    except MissingCollectionError:
        print("Writing into a new RUN collection")
        pass
    else:
        print("Prior RUN collection located and successfully removed")

    _ = writeable_butler.registry.registerCollection(si_input_collection, type=CollectionType.RUN)

Writing into a new RUN collection
Writing into a new RUN collection
Writing into a new RUN collection
Writing into a new RUN collection
Writing into a new RUN collection


In [11]:
for lim in [27,29,30,31,33]:
    for band in bands:
        dataset_ref_lim32 = si.ingest_injection_catalog(
            writeable_butler=writeable_butler,
            table=injection_cats[lim][band],
            band=band,
            output_collection=f"u/pferguso/full_dwarf_test/2_lim_{lim}",
            dataset_type_name='injection_catalog'
        )

In [14]:
# si_dataset_type =  DatasetType(
#     "si_cat",
#     dimensions=["skymap", "tract"],
#     storageClass="DataFrame",
#     universe=DimensionUniverse(),
# )

# writeable_butler.registry.registerDatasetType(si_dataset_type)

NameError: name 'writeable_butler' is not defined

In [29]:
for band in bands:
    dataset_ref_lim32 = si.ingest_injection_catalog(
        writeable_butler=writeable_butler,
        table=cat_lim32[band],
        band=band,
        output_collection="u/pferguso/full_dwarf_test/2_lim_32",
        dataset_type_name='injection_catalog'
    )

    dataset_ref_lim28 = si.ingest_injection_catalog(
        writeable_butler=writeable_butler,
        table=cat_lim28[band],
        band=band,
        output_collection="u/pferguso/full_dwarf_test/2_lim_28",
        dataset_type_name='injection_catalog'
    )

    print(f'{dataset_ref_lim32 = }')

dataset_ref_lim32 = [DatasetRef(DatasetType('injection_catalog', {band, htm7}, ArrowAstropy), {band: 'g', htm7: 165960}, run='u/pferguso/full_dwarf_test/2_lim_32', id=6e52a607-92e8-4073-baf1-833c06b579de), DatasetRef(DatasetType('injection_catalog', {band, htm7}, ArrowAstropy), {band: 'g', htm7: 165988}, run='u/pferguso/full_dwarf_test/2_lim_32', id=b21fe010-6d10-4a97-8df5-a574199f2826), DatasetRef(DatasetType('injection_catalog', {band, htm7}, ArrowAstropy), {band: 'g', htm7: 166004}, run='u/pferguso/full_dwarf_test/2_lim_32', id=1410fb95-1ed8-4080-8723-d00e3468923f)]


dataset_ref_lim32 = [DatasetRef(DatasetType('injection_catalog', {band, htm7}, ArrowAstropy), {band: 'r', htm7: 165960}, run='u/pferguso/full_dwarf_test/2_lim_32', id=385f99a7-ceba-43ba-9faa-526d5eb1ba21), DatasetRef(DatasetType('injection_catalog', {band, htm7}, ArrowAstropy), {band: 'r', htm7: 165988}, run='u/pferguso/full_dwarf_test/2_lim_32', id=7398257e-bdca-467d-94ae-50c119402670), DatasetRef(DatasetType('injection_catalog', {band, htm7}, ArrowAstropy), {band: 'r', htm7: 166004}, run='u/pferguso/full_dwarf_test/2_lim_32', id=e0847f4b-2dc0-4239-a6a7-0df8c573d86d)]


dataset_ref_lim32 = [DatasetRef(DatasetType('injection_catalog', {band, htm7}, ArrowAstropy), {band: 'i', htm7: 165960}, run='u/pferguso/full_dwarf_test/2_lim_32', id=0d81ff51-01ca-484e-ba74-a1d37b261c88), DatasetRef(DatasetType('injection_catalog', {band, htm7}, ArrowAstropy), {band: 'i', htm7: 165988}, run='u/pferguso/full_dwarf_test/2_lim_32', id=e348caf7-2269-4d28-803f-0b34ace3c8f9), DatasetRef(DatasetType('injection_catalog', {band, htm7}, ArrowAstropy), {band: 'i', htm7: 166004}, run='u/pferguso/full_dwarf_test/2_lim_32', id=200fc33f-f101-4f03-9f49-142aea199939)]
